In [0]:
import os
import torchvision.datasets
from torch.utils import data
from torchvision.datasets import utils
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.utils import save_image
import time
import os
import csv
import torchsummary
import io
from contextlib import redirect_stdout
from math import ceil
import torchvision.transforms as transforms

###definition de nos hyperparametres

In [0]:
#definition des hyperparametres
batch_size = 64
lr = 3e-4
epochs = 20
save_interval = 2
log_interval = 100
np.random.seed(1111)
torch.random.manual_seed(1111)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


#Fontion de Chargement de la base de donnees Street View House Numbers (SVHN) 
#de maisons house numbers de Google Street View
def loading_dataset_SVHN(dataset_location, batch_size):
    image_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((.5, .5, .5),
                             (.5, .5, .5))
    ])

    trainvalid = torchvision.datasets.SVHN(
        dataset_location, split='train',
        download=True,
        transform=image_transform
    )

    trainset_size = int(len(trainvalid) * 0.9)
    trainset, validset = data.dataset.random_split(
        trainvalid,
        [trainset_size, len(trainvalid) - trainset_size]
    )

    training_loader = data.DataLoader(
        trainset,
        batch_size=batch_size,	
        shuffle=True,
        num_workers=2
    )

    validation_loader = data.DataLoader(
        validset,
        batch_size=batch_size,
    )

    test_loader = data.DataLoader(
        torchvision.datasets.SVHN(
            dataset_location, split='test',
            download=True,
            transform=image_transform
        ),
        batch_size=batch_size,
    )

    return training_loader, validation_loader, test_loader


####definition du modele VAE Q.3 ici in_channels=3 car le dataset SVHN est de dimension 3x32x32

In [0]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(7, 7))
        self.pool1 = nn.AvgPool2d(kernel_size=(2, 2), stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3))
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2), stride=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=(5, 5))
        self._mean = nn.Linear(in_features=256, out_features=100, bias=True)
        self._logvar = nn.Linear(in_features=256, out_features=100, bias=True)
        self.elu = nn.ELU()
    def forward(self, input):
        input = self.conv1(input)
        input = self.elu(input)
        input = self.pool1(input)
        input = self.conv2(input)
        input = self.elu(input)
        input = self.pool2(input)
        input = self.conv3(input)
        input = self.elu(input)
        input = input.view(input.size(0), -1)
        mean = self._mean(input)
        logvar = self._logvar(input)
        return mean, logvar

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.linear = nn.Linear(in_features=100, out_features=256, bias=True)
        self.conv1 = nn.Conv2d(in_channels=256, out_channels=64, kernel_size=(5, 5), padding=4)
        self.upsample1 = nn.Upsample(scale_factor=2, mode='bilinear')
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(3, 3), padding=2)
        self.upsample2 = nn.Upsample(scale_factor=2, mode='bilinear')
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=(3, 3), padding=2)
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=3, kernel_size=(3, 3), padding=4)
        self.elu = nn.ELU()
    def forward(self, input):
        input = self.linear(input)
        input = input.unsqueeze(-1).unsqueeze(-1)
        input = self.elu(input)
        input = self.conv1(input)
        input = self.elu(input)
        input = self.upsample1(input)
        input = self.conv2(input)
        input = self.elu(input)
        input = self.upsample2(input)
        input = self.conv3(input)
        input = self.elu(input)
        input_tilde = self.conv4(input)
        return input_tilde


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        mean, logvar = self.encoder(x)
        epsilon = torch.randn_like(logvar)
        z = mean + torch.exp(logvar / 2) * epsilon
        input_tilde_logits = self.decoder(z)
        return input_tilde_logits, mean, logvar


#ELBO utilisant un decoder p(x|z) avec une distribution normale de variance 1
class Elbo(nn.Module):
    def __init__(self):
        super(Elbo, self).__init__()

    def forward(self, x, x_tilde, mean, logvar):
        x = x.reshape((x.shape[0],-1))
        x_tilde = x_tilde.reshape((x_tilde.shape[0], -1))
        kl = 0.5 * (-logvar - 1 + mean**2 + torch.exp(logvar)).sum(dim=-1)
        log_decoder = -0.5 * torch.sum((x - x_tilde)**2, -1) - 0.5 * x.shape[1] * np.log(2 * np.pi)
        loss = -(log_decoder - kl).mean()
        return loss


####Creation de la fonction de training (entrainement)

In [0]:
# retourner l'entrainement de chaque epoque(une epoch)
def epoch_training(model, optimizer, data_load, loss_function, epoch, log_interval=None):
    model.train()
    elbo = []
    for batch_id, (x, _) in enumerate(data_load):
        x = x.to(device)
        optimizer.zero_grad()
        x_tilde_logits, mean, logvar = model(x)
        loss = loss_function(x, x_tilde_logits, mean, logvar)
        loss.backward()
        optimizer.step()
        elbo.append(-loss.item())
        if log_interval is not None:
            if batch_id%log_interval == 0:
                print (f"----> Epoch {epoch},\t Batch {batch_id},\t Training ELBO: {np.mean(np.array(elbo)):.2f}")
    return np.mean(np.array(elbo))

# faire l'evaluation du modele dans les data 
def epoch_validation(model, data_load, loss_function):
    model.eval()
    elbo = []
    for batch_id, (x, _) in enumerate(data_load):
        x = x.to(device)
        x_tilde_logits, mean, logvar = model(x)
        loss = loss_function(x, x_tilde_logits, mean, logvar)
        elbo.append(-loss.item())
    return np.mean(np.array(elbo))

#sauvegarder le modele
def save_model(model, optimizer, train_elbos, val_elbos, epoch_time, epoch, save_dir, best_model=False):
    if best_model:
        path = os.path.join(save_dir, f'best_model.pth')
    else:
        path = os.path.join(save_dir, f'model_epoch_{epoch}.pth')
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_elbos': train_elbos,
        'val_elbos': val_elbos,
    }, path)

    epochs = [j for j in range(epoch+1)]
    stats = {'Epoch': epochs, 'Training Elbo': train_elbos, "Validation Elbo": val_elbos, "Epoch Time": epoch_time}
    stats_path = os.path.join(save_dir, 'Summary_stats.csv')
    with open(stats_path, 'w') as csvfile:
        fieldnames = stats.keys()
        writer = csv.writer(csvfile)
        writer.writerow(fieldnames)
        writer.writerows([[stats[key][j] for key in fieldnames] for j in epochs])



#Entrainement du modele pour un nombre d'epoque(epochs) definie
def train(model, optimizer, training_load, val_loader, loss_function, epochs, save_dir = os.curdir, save_interval=None, log_interval=None,
          model_outputs_logits=True, train_samples=None, val_samples=None, random_z=None):
    train_elbos = []
    val_elbos = []
    epoch_time = []
    max_val_elbo = -1000000
    for epoch in range(epochs):
        stime = time.time()
        train_elbo = epoch_training(model, optimizer, training_load, loss_function, epoch, log_interval)
        val_elbo = epoch_validation(model, val_loader, loss_function)
        train_elbos.append(train_elbo)
        val_elbos.append(val_elbo)
        epoch_time_ = time.time() - stime
        epoch_time.append(epoch_time_)

        if val_elbo > max_val_elbo:
            max_val_elbo = val_elbo
            if save_interval is not None:
                save_model(model, optimizer, train_elbos, val_elbos, epoch_time, epoch, save_dir, True)
                generer_echantillon(model, save_dir=save_dir, epoch=epoch, train_samples=train_samples,
                                 val_samples=val_samples, random_z=random_z, model_outputs_logits=model_outputs_logits, best=True)

        if save_interval is not None:
            if epoch % save_interval == 0:
                save_model(model, optimizer, train_elbos, val_elbos, epoch_time, epoch, save_dir, False)
                generer_echantillon(model, save_dir=save_dir, epoch=epoch, train_samples=train_samples,
                                 val_samples=val_samples, random_z=random_z, model_outputs_logits=model_outputs_logits)

        print(f"-> Epoch {epoch},\t Training ELBO: {train_elbo:.2f},\t Validation ELBO: {val_elbo:.2f},\t "
              f"Max Validation ELBO: {max_val_elbo:.2f},\t Epoch Time: {epoch_time_:.2f} seconds")
    if save_interval is not None:
        save_model(model, optimizer, train_elbos, val_elbos, epoch_time, epoch, save_dir, False)
        generer_echantillon(model, save_dir=save_dir, epoch=epoch, train_samples=train_samples,
                         val_samples=val_samples, random_z=random_z, model_outputs_logits=model_outputs_logits)
    return train_elbos, val_elbos, epoch_time[-1]


####Generate original samples vs reconstructed samples for the training and validation sets, plus some random samples

In [0]:

def generer_echantillon(model, save_dir, epoch, train_samples, val_samples, random_z, model_outputs_logits=True, best=False):
    with torch.no_grad():
        train_samples = train_samples.to(device)
        val_samples = val_samples.to(device)
        random_z = random_z.to(device)

        generated_train_samples, mean,_ = model(train_samples)
        generated_val_samples, _, _ = model(val_samples)
        generated_random_samples = model.decoder(random_z)

        if model_outputs_logits:
            generated_train_samples = (torch.sigmoid(generated_train_samples)).round().cpu()
            generated_val_samples = (torch.sigmoid(generated_val_samples)).round().cpu()
            generated_random_samples = (torch.sigmoid(generated_random_samples)).round().cpu()
        else: 
            generated_train_samples = ((generated_train_samples + 1) / 2).cpu()
            generated_val_samples = ((generated_val_samples + 1) / 2).cpu()
            generated_random_samples = ((generated_random_samples + 1) / 2).cpu()

    if epoch == 0:
        if model_outputs_logits:
            save_image(train_samples, os.path.join(save_dir, "input_training_samples.png"))
            save_image(val_samples, os.path.join(save_dir, "input_validation_samples.png"))
        else:  
            save_image((train_samples + 1) / 2, os.path.join(save_dir, "input_training_samples.png"))
            save_image((val_samples + 1) / 2, os.path.join(save_dir, "input_validation_samples.png"))

    if best:
        generated_train_path = os.path.join(save_dir, f"generer_train_samples_best.png")
        generated_val_path = os.path.join(save_dir, f"generer_val_samples_best.png")
        generated_random_path = os.path.join(save_dir, f"generer_random_samples_best.png")
    else:
        generated_train_path = os.path.join(save_dir, f"generer_train_samples_{epoch}.png")
        generated_val_path = os.path.join(save_dir, f"generer_val_samples_{epoch}.png")
        generated_random_path = os.path.join(save_dir, f"generer_random_samples_{epoch}.png")

    save_image(generated_train_samples, generated_train_path)
    save_image(generated_val_samples, generated_val_path)
    save_image(generated_random_samples, generated_random_path)


In [9]:
#Chargement de la base de donnees Street View House Numbers (SVHN) venant de numero de maisons house numbers de Google Street View
training_loader, validation_loader, test_loader = loading_dataset_SVHN('/content/data', batch_size=batch_size)

save_parent_path = '/content'
model_folder = "Q3_VAE_MODEL_"
previous_folders = [f for f in os.listdir(save_parent_path) if (model_folder in f and os.path.isdir(f))]
if not previous_folders:
    save_dir = os.path.join(save_parent_path, model_folder + f'{1:03d}')
else:
    last_model = max(previous_folders)
    i = int(last_model.replace(model_folder,"")) + 1
    save_dir = os.path.join(save_parent_path, model_folder + f'{i:03d}')
os.mkdir(save_dir)

#Creons notre modele
model = VAE()
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = Elbo()


train_samples, _ = next(iter(training_loader))
val_samples, _ = next(iter(validation_loader))
random_z = torch.randn((batch_size, 100))


#Entrainons notre modele
train_elbos, val_elbos, epoch_time = train(model, optimizer, training_loader, validation_loader, loss_fn, epochs=epochs,
                                           save_dir=save_dir, save_interval=save_interval, log_interval=log_interval, 
                                           model_outputs_logits=False, train_samples=train_samples, val_samples=val_samples, random_z=random_z)


Using downloaded and verified file: /content/data/train_32x32.mat
Using downloaded and verified file: /content/data/test_32x32.mat


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


----> Epoch 0,	 Batch 0,	 Training ELBO: -3066.36
----> Epoch 0,	 Batch 100,	 Training ELBO: -2995.45
----> Epoch 0,	 Batch 200,	 Training ELBO: -2979.68
----> Epoch 0,	 Batch 300,	 Training ELBO: -2970.53
----> Epoch 0,	 Batch 400,	 Training ELBO: -2964.73
----> Epoch 0,	 Batch 500,	 Training ELBO: -2959.92
----> Epoch 0,	 Batch 600,	 Training ELBO: -2956.29
----> Epoch 0,	 Batch 700,	 Training ELBO: -2953.55
----> Epoch 0,	 Batch 800,	 Training ELBO: -2951.23
----> Epoch 0,	 Batch 900,	 Training ELBO: -2949.34
----> Epoch 0,	 Batch 1000,	 Training ELBO: -2947.75
-> Epoch 0,	 Training ELBO: -2947.28,	 Validation ELBO: -2934.82,	 Max Validation ELBO: -2934.82,	 Epoch Time: 21.41 seconds
----> Epoch 1,	 Batch 0,	 Training ELBO: -2943.72
----> Epoch 1,	 Batch 100,	 Training ELBO: -2931.77
----> Epoch 1,	 Batch 200,	 Training ELBO: -2931.94
----> Epoch 1,	 Batch 300,	 Training ELBO: -2932.38
----> Epoch 1,	 Batch 400,	 Training ELBO: -2932.07
----> Epoch 1,	 Batch 500,	 Training ELBO: -29

#### !rm -rf '/content/Q3_VAE_MODEL_009'
!rm -rf '/content/Q3_VAE_MODEL_002/representation_echantillon_pertubation'

####Question 3.2 le modele VAE

In [0]:
#produire un nbre echantillon des echantillons aleatoire
def generate_random_samples(model, save_dir, random_z_samples=None, epoch=-1, num_samples=200, latent_size=100,
                            model_outputs_logits=True, samples_per_image=64, generated_random_file="Echant_aleatoire"):
    with torch.no_grad():
        num_images = ceil(num_samples / samples_per_image)
        for k in range(1, num_images +1):
            if k == num_images and num_samples%samples_per_image != 0:
                samples_per_image = num_samples%samples_per_image
            if random_z_samples is None:
                random_z = torch.randn((samples_per_image, latent_size))
            else:
                try:
                    random_z = random_z_samples[(k-1)*samples_per_image:k*samples_per_image]
                except:
                    if len(random_z_samples) < k*samples_per_image:
                        print("echantillon aleatoire de z est tres petit pour le nbre d'images demandes")
            random_z = random_z.to(device)
            generated_random_samples = model.decoder(random_z)
            if model_outputs_logits:
                generated_random_samples = (torch.sigmoid(generated_random_samples)).round().cpu()
            else: 
                generated_random_samples = ((generated_random_samples + 1) / 2).cpu()
            generated_random_file_ = generated_random_file + f"_epoch_{epoch}_echant_{num_samples}_{k:03d}.png"
            image_path = os.path.join(save_dir, generated_random_file_)
            save_image(generated_random_samples, image_path)

In [11]:
random_z_samples_ = torch.randn((64, 100))
epsilon = 1.5
save_dir_disentangled = os.path.join(save_dir, "echantillon_pertubation") 
if not os.path.isdir(save_dir_disentangled):
  os.mkdir(save_dir_disentangled)
  generate_random_samples(model, save_dir_disentangled, random_z_samples=random_z_samples_, epoch=20, num_samples=64, latent_size=100,
                            model_outputs_logits=False, samples_per_image=64, generated_random_file="sans_pertubation")
for i in range(50):
  random_z_samples = random_z_samples_
  random_z_samples[:,i] += epsilon
  generate_random_samples(model, save_dir_disentangled, random_z_samples=random_z_samples, epoch=20, num_samples=64, latent_size=100,
                            model_outputs_logits=False, samples_per_image=64, generated_random_file=f"avec_pertubation_{i}")

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


#### question Q3.3 VAE

In [0]:
#generons les images interpolees random_z_1 and random_z_2 avec un facteur
def generer_echantillon_interpole(model, save_dir, alphas, interpolate_images=False, random_z_0=None, random_z_1=None, epoch=-1, num_samples=64, latent_size=100,
                            model_outputs_logits=False, generated_random_file="random_interpolation"):
    save_dir = os.path.join(save_dir, "Echantillon_interpolation")
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    with torch.no_grad():
        if (random_z_0 is None) or (random_z_1 is None):
            random_z_0 = torch.randn((num_samples, latent_size))
            random_z_1 = torch.randn((num_samples, latent_size))
        random_z_0 = random_z_0.to(device)
        random_z_1 = random_z_1.to(device)
        if interpolate_images:
            generated_images_0 = model.decoder(random_z_0)
            generated_images_1 = model.decoder(random_z_1)
            for alpha in alphas:
                generated_images = alpha*generated_images_0 + (1.-alpha) * generated_images_1
                if model_outputs_logits:
                    generated_images = (torch.sigmoid(generated_images)).round().cpu()
                else: 
                    generated_images = ((generated_images + 1) / 2).cpu()
                generated_random_file_ = generated_random_file + f"_images_alpha_{alpha}.png"
                image_path = os.path.join(save_dir, generated_random_file_)
                save_image(generated_images, image_path)
        else:
            for alpha in alphas:
                random_z = alpha*random_z_0 + (1.-alpha)*random_z_1
                generated_images = model.decoder(random_z)
                if model_outputs_logits:
                    generated_images = (torch.sigmoid(generated_images)).round().cpu()
                else:  
                    generated_images = ((generated_images + 1) / 2).cpu()
                generated_random_file_ = generated_random_file + f"_latent_alpha_{alpha}.png"
                image_path = os.path.join(save_dir, generated_random_file_)
                save_image(generated_images, image_path)


In [13]:

random_z_0 = torch.randn((64, 100))
random_z_1 = torch.randn((64, 100))
alphas = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
generer_echantillon_interpole(model, save_dir, alphas, interpolate_images=False, random_z_0=random_z_0, random_z_1=random_z_1, 
                              epoch=20, num_samples=64, latent_size=100, model_outputs_logits=False)
generer_echantillon_interpole(model, save_dir, alphas, interpolate_images=True, random_z_0=random_z_0, random_z_1=random_z_1, 
                              epoch=20, num_samples=64, latent_size=100, model_outputs_logits=False)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


####Q3.4 valuation quantitative

In [14]:
file="Echant_aleatoire"

for i in range(1000):
  random_z = torch.randn((1, 100))
  random_z = random_z.to(device)
  random_samples = model.decoder(random_z)
  file_ = os.path.join(save_dir, f"eval_image_{i}.png")
  image_path = os.path.join(save_dir, file_)
  save_image(random_samples, image_path)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


  !zip -r dossier1.zip  /content/Q3_VAE_MODEL_002